https://keras.io/examples/keras_recipes/bayesian_neural_networks/

In [1]:
!pip install tensorflow-probability
!pip install tensorflow-datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import tensorflow_probability as tfp

In [7]:
def get_train_and_test_splits(train_size, batch_size=1):
    # We prefetch with a buffer the same size as the dataset because the dataset
    # is very small and fits into memory.
    dataset = (tfds.load(name="wine_quality", as_supervised=True, split="train").map(lambda x, y: (x, tf.cast(y, tf.float32)))
                                                                                .prefetch(buffer_size=dataset_size)
                                                                                .cache())
    # We shuffle with a buffer the same size as the dataset.
    train_dataset = (dataset.take(train_size).shuffle(buffer_size=train_size).batch(batch_size))
    test_dataset = dataset.skip(train_size).batch(batch_size)

    return train_dataset, test_dataset

In [9]:
hidden_units = [8, 8]
learning_rate = 0.001

def run_experiment(model, loss, train_dataset, test_dataset):
    model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=learning_rate),
                  loss=loss, metrics=[keras.metrics.RootMeanSquaredError()])

    print("Start training the model...")
    model.fit(train_dataset, epochs=num_epochs, validation_data=test_dataset)
    print("Model training finished.")
    _, rmse = model.evaluate(train_dataset, verbose=0)
    print(f"Train RMSE: {round(rmse, 3)}")

    print("Evaluating model performance...")
    _, rmse = model.evaluate(test_dataset, verbose=0)
    print(f"Test RMSE: {round(rmse, 3)}")

In [10]:
FEATURE_NAMES = ["fixed acidity", "volatile acidity", "citric acid",
                 "residual sugar", "chlorides", "free sulfur dioxide",
                 "total sulfur dioxide", "density", "pH", "sulphates", "alcohol"]

def create_model_inputs():
    inputs = {}
    for feature_name in FEATURE_NAMES:
        inputs[feature_name] = layers.Input(name=feature_name, 
                                            shape=(1,), dtype=tf.float32)
    return inputs

In [11]:
def create_baseline_model():
    inputs = create_model_inputs()
    input_values = [value for _, value in sorted(inputs.items())]
    features = keras.layers.concatenate(input_values)
    features = layers.BatchNormalization()(features)

    # Create hidden layers with deterministic weights using the Dense layer.
    for units in hidden_units:
        features = layers.Dense(units, activation="sigmoid")(features)
    # The output is deterministic: a single point estimate.
    outputs = layers.Dense(units=1)(features)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [12]:
dataset_size = 4898
batch_size = 256
train_size = int(dataset_size * 0.85)
train_dataset, test_dataset = get_train_and_test_splits(train_size, batch_size)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/4898 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/wine_quality/white/1.0.0.incompleteK50L94/wine_quality-train.tfrecord*...:   0…

Dataset wine_quality downloaded and prepared to ~/tensorflow_datasets/wine_quality/white/1.0.0. Subsequent calls will reuse this data.


In [13]:
num_epochs = 100
mse_loss = keras.losses.MeanSquaredError()
baseline_model = create_baseline_model()
run_experiment(baseline_model, mse_loss, train_dataset, test_dataset)

Start training the model...
Epoch 1/100
17/17 [==============================] - 2s 48ms/step - loss: 26.7321 - root_mean_squared_error: 5.1703 - val_loss: 24.0098 - val_root_mean_squared_error: 4.9000
Epoch 2/100
17/17 [==============================] - 0s 8ms/step - loss: 24.8487 - root_mean_squared_error: 4.9848 - val_loss: 22.6761 - val_root_mean_squared_error: 4.7619
Epoch 3/100
17/17 [==============================] - 0s 8ms/step - loss: 23.2236 - root_mean_squared_error: 4.8191 - val_loss: 21.4194 - val_root_mean_squared_error: 4.6281
Epoch 4/100
17/17 [==============================] - 0s 8ms/step - loss: 21.6516 - root_mean_squared_error: 4.6531 - val_loss: 20.1809 - val_root_mean_squared_error: 4.4923
Epoch 5/100
17/17 [==============================] - 0s 8ms/step - loss: 20.1098 - root_mean_squared_error: 4.4844 - val_loss: 18.9066 - val_root_mean_squared_error: 4.3482
Epoch 6/100
17/17 [==============================] - 0s 7ms/step - loss: 18.5998 - root_mean_squared_error

In [14]:
sample = 10
examples, targets = list(test_dataset.unbatch().shuffle(batch_size * 10)
                                               .batch(sample))[0]

predicted = baseline_model(examples).numpy()
for idx in range(sample):
    print(f"Predicted: {round(float(predicted[idx][0]), 1)} - Actual: {targets[idx]}")

Predicted: 5.6 - Actual: 5.0
Predicted: 5.9 - Actual: 5.0
Predicted: 5.6 - Actual: 5.0
Predicted: 6.6 - Actual: 7.0
Predicted: 5.3 - Actual: 5.0
Predicted: 5.7 - Actual: 5.0
Predicted: 6.3 - Actual: 6.0
Predicted: 4.9 - Actual: 4.0
Predicted: 5.6 - Actual: 6.0
Predicted: 6.1 - Actual: 6.0


In [16]:
# Define the prior weight distribution as Normal of mean=0 and stddev=1.
# Note that, in this example, the prior distribution is not trainable,
# as we fix its parameters.
def prior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    prior_model = keras.Sequential([tfp.layers.DistributionLambda(
        lambda t: tfp.distributions.MultivariateNormalDiag(loc=tf.zeros(n), scale_diag=tf.ones(n)))])
    return prior_model

# Define variational posterior weight distribution as multivariate Gaussian.
# Note that the learnable parameters for this distribution are the means,
# variances, and covariances.
def posterior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    posterior_model = keras.Sequential([tfp.layers.VariableLayer(
        tfp.layers.MultivariateNormalTriL.params_size(n), dtype=dtype),
        tfp.layers.MultivariateNormalTriL(n)])
    return posterior_model

In [17]:
def create_bnn_model(train_size):
    inputs = create_model_inputs()
    features = keras.layers.concatenate(list(inputs.values()))
    features = layers.BatchNormalization()(features)

    # Create hidden layers with weight uncertainty using the DenseVariational layer.
    for units in hidden_units:
        features = tfp.layers.DenseVariational(units=units, make_prior_fn=prior,
                                               make_posterior_fn=posterior, kl_weight=1 / train_size,
                                               activation="sigmoid")(features)

    # The output is deterministic: a single point estimate.
    outputs = layers.Dense(units=1)(features)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [18]:
num_epochs = 500
train_sample_size = int(train_size * 0.3)
small_train_dataset = train_dataset.unbatch().take(train_sample_size).batch(batch_size)

bnn_model_small = create_bnn_model(train_sample_size)
run_experiment(bnn_model_small, mse_loss, small_train_dataset, test_dataset)

Start training the model...
Epoch 1/500
5/5 [==============================] - 3s 293ms/step - loss: 35.1559 - root_mean_squared_error: 5.9279 - val_loss: 30.3990 - val_root_mean_squared_error: 5.5124
Epoch 2/500
5/5 [==============================] - 0s 24ms/step - loss: 34.3448 - root_mean_squared_error: 5.8594 - val_loss: 36.0074 - val_root_mean_squared_error: 5.9990
Epoch 3/500
5/5 [==============================] - 0s 24ms/step - loss: 34.1548 - root_mean_squared_error: 5.8429 - val_loss: 31.8737 - val_root_mean_squared_error: 5.6442
Epoch 4/500
5/5 [==============================] - 0s 22ms/step - loss: 33.3559 - root_mean_squared_error: 5.7743 - val_loss: 31.8844 - val_root_mean_squared_error: 5.6455
Epoch 5/500
5/5 [==============================] - 0s 24ms/step - loss: 33.1291 - root_mean_squared_error: 5.7546 - val_loss: 31.2953 - val_root_mean_squared_error: 5.5926
Epoch 6/500
5/5 [==============================] - 0s 24ms/step - loss: 29.8024 - root_mean_squared_error: 5.45

In [19]:
def compute_predictions(model, iterations=100):
    predicted = []
    for _ in range(iterations):
        predicted.append(model(examples).numpy())
    predicted = np.concatenate(predicted, axis=1)

    prediction_mean = np.mean(predicted, axis=1).tolist()
    prediction_min = np.min(predicted, axis=1).tolist()
    prediction_max = np.max(predicted, axis=1).tolist()
    prediction_range = (np.max(predicted, axis=1) - np.min(predicted, axis=1)).tolist()

    for idx in range(sample):
        print(f"Predictions mean: {round(prediction_mean[idx], 2)}, "
              f"min: {round(prediction_min[idx], 2)}, "
              f"max: {round(prediction_max[idx], 2)}, "
              f"range: {round(prediction_range[idx], 2)} - "
              f"Actual: {targets[idx]}")

compute_predictions(bnn_model_small)

Predictions mean: 5.59, min: 4.99, max: 6.03, range: 1.04 - Actual: 5.0
Predictions mean: 5.92, min: 5.37, max: 6.31, range: 0.94 - Actual: 5.0
Predictions mean: 5.52, min: 4.92, max: 6.06, range: 1.15 - Actual: 5.0
Predictions mean: 6.37, min: 5.96, max: 6.57, range: 0.61 - Actual: 7.0
Predictions mean: 5.23, min: 4.52, max: 5.98, range: 1.46 - Actual: 5.0
Predictions mean: 5.69, min: 5.07, max: 6.1, range: 1.03 - Actual: 5.0
Predictions mean: 6.24, min: 5.81, max: 6.47, range: 0.67 - Actual: 6.0
Predictions mean: 5.28, min: 4.46, max: 5.97, range: 1.51 - Actual: 4.0
Predictions mean: 5.64, min: 4.98, max: 6.1, range: 1.12 - Actual: 6.0
Predictions mean: 6.11, min: 5.64, max: 6.47, range: 0.84 - Actual: 6.0


In [20]:
num_epochs = 500
bnn_model_full = create_bnn_model(train_size)
run_experiment(bnn_model_full, mse_loss, train_dataset, test_dataset)

compute_predictions(bnn_model_full)

Start training the model...
Epoch 1/500
17/17 [==============================] - 3s 51ms/step - loss: 35.7509 - root_mean_squared_error: 5.9788 - val_loss: 31.3070 - val_root_mean_squared_error: 5.5949
Epoch 2/500
17/17 [==============================] - 0s 9ms/step - loss: 36.5466 - root_mean_squared_error: 6.0450 - val_loss: 34.6630 - val_root_mean_squared_error: 5.8871
Epoch 3/500
17/17 [==============================] - 0s 9ms/step - loss: 34.7357 - root_mean_squared_error: 5.8933 - val_loss: 34.8302 - val_root_mean_squared_error: 5.9014
Epoch 4/500
17/17 [==============================] - 0s 9ms/step - loss: 31.5025 - root_mean_squared_error: 5.6123 - val_loss: 31.0020 - val_root_mean_squared_error: 5.5676
Epoch 5/500
17/17 [==============================] - 0s 9ms/step - loss: 30.4225 - root_mean_squared_error: 5.5152 - val_loss: 28.3790 - val_root_mean_squared_error: 5.3267
Epoch 6/500
17/17 [==============================] - 0s 9ms/step - loss: 30.4557 - root_mean_squared_error